# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

The dataset is extracted from an Udacity's competition called 'Predicting customer satisfaction' It consists of customers data from Santander bank, containing all their relevant features (like the number of products, their name, interest rate of loans, etc). 

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()
cluster_name = 'project-cluster'
try:
    compute_target = ComputeTarget(ws, cluster_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', min_nodes=1, max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

In [1]:
#import dataset
from azureml.data.dataset_factory import TabularDatasetFactory
Data_URL= 'https://www.kaggle.com/keplersmachines/kepler-labelled-time-series-data/download'
ds=TabularDatasetFactory.from_delimited_files(path=Data_URL)
# choose a name for experiment
experiment_name = 'exp-capstone'
experiment=Experiment(ws, experiment_name)

SyntaxError: invalid syntax (<ipython-input-1-d987212cca63>, line 2)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

- Since the dataset is unbalanced (HOW), F1-score is appropriate given that it takes into account both precision and recall.

In [ ]:
# TODO: Put your automl settings here
automl_settings = {
       "n_cross_validations": 3,
       "primary_metric": 'F1_score',
       "enable_early_stopping": True,
       "experiment_timeout_hours": 1.0,
       "max_concurrent_iterations": 4,
       "max_cores_per_iteration": -1,
       "verbosity": logging.INFO,
   }
# TODO: Put your automl config here
automl_config = AutoMLConfig(task = 'classification',
                               compute_target = compute_target,
                               training_data = ds,
                               label_column_name = "LABEL",
                               **automl_settings
                               )

In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_run, best_model = remote_run.get_output()

In [ ]:
#TODO: Save the best model
best_run.register_model(model_name = "model.pkl", model_path = './outputs/')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service